In [1]:
from model import CNN_Encoder,RNN_Decoder
import tensorflow as tf
embedding_dim = 256 #임베딩 차원
units = 512 #노드 개수는 512개
vocab_size = 10000 + 1 #사전크기 50000개 

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()


#체크포인트 불러오기
checkpoint_path = "./checkpoints/final_model1"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)



In [4]:
import tensorflow as tf
import pickle
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
    
with open('./tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('./train_captions.pkl', 'rb') as f:
    train_captions = pickle.load(f)

In [5]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

train_seqs = tokenizer.texts_to_sequences(train_captions)
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
max_length = calc_max_length(train_seqs)

In [6]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

def evaluate(image):
    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result

        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [7]:
evaluate("/home/seeds/iconms12/tmp/tf_image_captioning/robber_img (5).jpg")

['a',
 'man',
 'making',
 'a',
 'skateboard',
 'typing',
 'on',
 'and',
 'her',
 'feet',
 'in',
 'a',
 'small',
 'stroller',
 '<end>']

In [8]:
#번역
import requests
import re
import pandas as pd
import urllib.request
import urllib.parse
import ast

def trans(msg):
    client_id = "mY5TpelRyR7Rq2AE0muE"
    client_secret = "Gm46UShi6x"
    request_url = "https://openapi.naver.com/v1/papago/n2mt"
    headers = {"X-Naver-Client-Id": client_id , "X-Naver-Client-Secret": client_secret }
    response = requests.post(request_url, headers=headers, data={"source": "en", "target": "ja", "text": msg})
    response = requests.post(request_url, headers=headers, data={"source": "ja", "target": "ko", "text": response.json()['message']['result']['translatedText']})
    result = response.json()['message']['result']['translatedText']
    
    return result#(re.sub("[a-zA-Z]","",tmp).strip())

In [9]:
trans("apple")

'사과'

In [41]:
#전송
import sys
from sdk.api.message import Message
from sdk.exceptions import CoolsmsException
from datetime import datetime

##  @brief This sample code demonstrate how to send sms through CoolSMS Rest API PHP
def send_sms(msg):
    now= datetime.now()
    # set api key, api secret
    api_key = "NCSGEF4CCDTGG1FM"
    api_secret = "3XA0AIA4N3JKBVOI6JA5RZMLC5ZGVWRH"

    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params['LMS'] = 'sms' # Message type ( sms, lms, mms, ata )
    params['to'] = '01088362658' # Recipients Number '01000000000,01000000001'
    params['from'] = '01088362658' # Sender number
    dt_string = f"{now.year}년{now.month}월 {now.day}일 {now.hour:02}시 {now.minute:02}분 {now.second:02}초"
    result="현재위치: 손민성 집의 CCTV \n 촬영시각: {} \n 상세내용: {}".format(dt_string,msg)
    params['text'] = result # Message

    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response['success_count'])
        print("Error Count : %s" % response['error_count'])
        print("Group ID : %s" % response['group_id'])

        if "error_list" in response:
            print("Error List : %s" % response['error_list'])

    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)

In [36]:
from socket import *
import struct
import shutil
import time
HOST = '210.115.229.4'
PORT = 9003
ADDR = (HOST, PORT)
BUFF_SIZE = 1024
FILE_NAME = './webcam_transfer/clientFile.jpg'
src="/home/seeds/iconms12/tmp/tf_image_captioning/webcam_transfer/clientFile.jpg"
dst="/home/seeds/iconms12/tmp/tf_image_captioning/detect_annomaly/"


serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(ADDR)
serverSocket.listen(5)

print("접속대기중")
clientSocket, addr = serverSocket.accept()
print("접속완료")    
count=1
while True:
    FILE_SIZE = clientSocket.recv(8)
    print(FILE_SIZE)
    #ime.sleep(0.001)
    FILE_SIZE = FILE_SIZE.decode()
    FILE_LEN = 0
    f = open(FILE_NAME, 'wb')
    while True:
        client_file = clientSocket.recv(BUFF_SIZE)
        # print(client_file)

        if not client_file:
            break

        f.write(client_file)
        FILE_LEN += len(client_file)

        if FILE_LEN == int(FILE_SIZE):
            break
    f.close()
    print('client : ' + FILE_NAME + ' file transfer')
    image="/home/seeds/iconms12/tmp/tf_image_captioning/webcam_transfer/clientFile.jpg"
    server_msg = evaluate(image) #캡션 전송
    sms_msg=' '.join(server_msg[:-1])
    if 'knife' in server_msg or 'sword' in server_msg or 'scissors' in server_msg:
        shutil.copy2(src,dst+f"detect_knife_{count}.jpg")
        count+=1
        msg=trans(sms_msg)
        send_sms(msg)
    # server_msg = bytes(server_msg, encoding='utf-8')
    # clientSocket.send(server_msg)
    clientSocket.send(sms_msg.encode())

clientSocket.close()

접속대기중
접속완료
b'70972'
client : ./webcam_transfer/clientFile.jpg file transfer
b'71225'
client : ./webcam_transfer/clientFile.jpg file transfer
b'68582'
client : ./webcam_transfer/clientFile.jpg file transfer
b'59072'
client : ./webcam_transfer/clientFile.jpg file transfer
b'55516'
client : ./webcam_transfer/clientFile.jpg file transfer
b'51128'
client : ./webcam_transfer/clientFile.jpg file transfer
b'50424'
client : ./webcam_transfer/clientFile.jpg file transfer
b'49854'
client : ./webcam_transfer/clientFile.jpg file transfer
b'52548'
client : ./webcam_transfer/clientFile.jpg file transfer
b'66972'
client : ./webcam_transfer/clientFile.jpg file transfer
b'60167'
client : ./webcam_transfer/clientFile.jpg file transfer
b'58111'
client : ./webcam_transfer/clientFile.jpg file transfer
b'59782'
client : ./webcam_transfer/clientFile.jpg file transfer
b'58244'
client : ./webcam_transfer/clientFile.jpg file transfer
b'58847'
client : ./webcam_transfer/clientFile.jpg file transfer
b'49826'
clien

b'54649'
client : ./webcam_transfer/clientFile.jpg file transfer
b'54405'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53282'
client : ./webcam_transfer/clientFile.jpg file transfer
b'54146'
client : ./webcam_transfer/clientFile.jpg file transfer
b'54729'
client : ./webcam_transfer/clientFile.jpg file transfer
b'54248'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53157'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53193'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53499'
client : ./webcam_transfer/clientFile.jpg file transfer
b'55715'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53896'
client : ./webcam_transfer/clientFile.jpg file transfer
b'53254'
client : ./webcam_transfer/clientFile.jpg file transfer
b'60617'
client : ./webcam_transfer/clientFile.jpg file transfer
b'63244'
client : ./webcam_transfer/clientFile.jpg file transfer
b'72664'
client : ./webcam_transfer/clientFile.jpg file transfer
b'67183'
client : ./webca

b'65496'
client : ./webcam_transfer/clientFile.jpg file transfer
b'65575'
client : ./webcam_transfer/clientFile.jpg file transfer
b'86901'
client : ./webcam_transfer/clientFile.jpg file transfer
b'88039'
client : ./webcam_transfer/clientFile.jpg file transfer
b'86602'
client : ./webcam_transfer/clientFile.jpg file transfer
b'52590'
client : ./webcam_transfer/clientFile.jpg file transfer
b'59951'
client : ./webcam_transfer/clientFile.jpg file transfer
b'61871'
client : ./webcam_transfer/clientFile.jpg file transfer
b'47581'
client : ./webcam_transfer/clientFile.jpg file transfer
b'40085'
client : ./webcam_transfer/clientFile.jpg file transfer
b'38954'
client : ./webcam_transfer/clientFile.jpg file transfer
b'38524'
client : ./webcam_transfer/clientFile.jpg file transfer
b'39278'
client : ./webcam_transfer/clientFile.jpg file transfer
b'41114'
client : ./webcam_transfer/clientFile.jpg file transfer
b'40091'
client : ./webcam_transfer/clientFile.jpg file transfer
b'39952'
client : ./webca

InvalidArgumentError: Expected image (JPEG, PNG, or GIF), got empty file [Op:DecodeJpeg]

In [31]:
f.close()

In [32]:
serverSocket.close()
clientSocket.close()